In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
  import shutil
  import os

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')


def setup_dataset(text):
  %cd /content/gdrive/My Drive/Kaggle/
  strings = text.split(' ')[-1]
  folder = strings.split('/')[1]
  os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle/"+folder
  print(strings)
  print(folder)
  !mkdir $folder
  shutil.copy2("kaggle.json","./"+folder+"/kaggle.json")
  %cd $folder
  !kaggle datasets download -d $strings
  
setup_dataset("kaggle datasets download -d ajaypalsinghlo/world-happiness-report-2021")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Kaggle
ajaypalsinghlo/world-happiness-report-2021
world-happiness-report-2021
mkdir: cannot create directory ‘world-happiness-report-2021’: File exists
/content/gdrive/My Drive/Kaggle/world-happiness-report-2021
world-happiness-report-2021.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
%ls

kaggle.json                      world-happiness-report-2021.zip
world-happiness-report-2021.csv  world-happiness-report.csv


In [5]:
!unzip world-happiness-report-2021.zip

Archive:  world-happiness-report-2021.zip
replace world-happiness-report-2021.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [6]:
df = pd.read_csv("world-happiness-report-2021.csv")

In [7]:
df.sort_values(by = ["Explained by: Freedom to make life choices"],ascending = False)

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
41,Uzbekistan,Commonwealth of Independent States,6.179,0.068,6.312,6.045,8.836,0.918,65.255,0.970,0.311,0.515,2.43,0.769,1.027,0.528,0.716,0.391,0.271,2.477
5,Norway,Western Europe,7.392,0.035,7.462,7.323,11.053,0.954,73.300,0.960,0.093,0.270,2.43,1.543,1.108,0.782,0.703,0.249,0.427,2.580
113,Cambodia,Southeast Asia,4.830,0.067,4.963,4.698,8.360,0.765,62.000,0.959,0.034,0.843,2.43,0.603,0.680,0.426,0.702,0.210,0.061,2.148
3,Iceland,Western Europe,7.554,0.059,7.670,7.438,10.878,0.983,73.000,0.955,0.160,0.673,2.43,1.482,1.172,0.772,0.698,0.293,0.170,2.967
0,Finland,Western Europe,7.842,0.032,7.904,7.780,10.775,0.954,72.000,0.949,-0.098,0.186,2.43,1.446,1.106,0.741,0.691,0.124,0.481,3.253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,Madagascar,Sub-Saharan Africa,4.208,0.072,4.349,4.068,7.396,0.686,59.305,0.552,-0.005,0.803,2.43,0.266,0.503,0.341,0.207,0.185,0.087,2.620
130,Comoros,Sub-Saharan Africa,4.289,0.084,4.454,4.123,8.031,0.626,57.349,0.548,0.082,0.781,2.43,0.488,0.367,0.279,0.202,0.241,0.101,2.610
122,Lebanon,Middle East and North Africa,4.584,0.055,4.691,4.477,9.626,0.848,67.355,0.525,-0.073,0.898,2.43,1.045,0.868,0.595,0.175,0.140,0.026,1.736
108,Algeria,Middle East and North Africa,4.887,0.053,4.991,4.783,9.342,0.802,66.005,0.480,-0.067,0.752,2.43,0.946,0.765,0.552,0.119,0.144,0.120,2.242


In [8]:
df.head()

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,Finland,Western Europe,7.842,0.032,7.904,7.780,10.775,0.954,72.0,0.949,-0.098,0.186,2.43,1.446,1.106,0.741,0.691,0.124,0.481,3.253
1,Denmark,Western Europe,7.620,0.035,7.687,7.552,10.933,0.954,72.7,0.946,0.030,0.179,2.43,1.502,1.108,0.763,0.686,0.208,0.485,2.868
2,Switzerland,Western Europe,7.571,0.036,7.643,7.500,11.117,0.942,74.4,0.919,0.025,0.292,2.43,1.566,1.079,0.816,0.653,0.204,0.413,2.839
3,Iceland,Western Europe,7.554,0.059,7.670,7.438,10.878,0.983,73.0,0.955,0.160,0.673,2.43,1.482,1.172,0.772,0.698,0.293,0.170,2.967
4,Netherlands,Western Europe,7.464,0.027,7.518,7.410,10.932,0.942,72.4,0.913,0.175,0.338,2.43,1.501,1.079,0.753,0.647,0.302,0.384,2.798


In [9]:
df[np.array(df.loc[:,"Country name"] == "India")]

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
138,India,South Asia,3.819,0.026,3.869,3.769,8.755,0.603,60.633,0.893,0.089,0.774,2.43,0.741,0.316,0.383,0.622,0.246,0.106,1.405


In [10]:
x = df.drop(["Explained by: Generosity"],axis = 1)
y = df["Explained by: Generosity"]

In [11]:
x = x.drop(["Country name","Regional indicator"],axis = 1)

,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Perceptions of corruption,Dystopia + residual
0,7.842,0.032,7.904,7.780,10.775,0.954,72.0,0.949,-0.098,0.186,2.43,1.446,1.106,0.741,0.691,0.481,3.253
1,7.620,0.035,7.687,7.552,10.933,0.954,72.7,0.946,0.030,0.179,2.43,1.502,1.108,0.763,0.686,0.485,2.868
2,7.571,0.036,7.643,7.500,11.117,0.942,74.4,0.919,0.025,0.292,2.43,1.566,1.079,0.816,0.653,0.413,2.839
3,7.554,0.059,7.670,7.438,10.878,0.983,73.0,0.955,0.160,0.673,2.43,1.482,1.172,0.772,0.698,0.170,2.967
4,7.464,0.027,7.518,7.410,10.932,0.942,72.4,0.913,0.175,0.338,2.43,1.501,1.079,0.753,0.647,0.384,2.798


##Train Test Split

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2)

##Standard Scaler

In [14]:
from sklearn.preprocessing import StandardScaler

In [15]:
sc = StandardScaler()

In [16]:
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

##Model

In [17]:
from tensorflow import keras

In [18]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout
from keras.layers import Activation

In [19]:
from keras.activations import relu,selu,linear,softplus

In [20]:
mle = keras.losses.MeanSquaredLogarithmicError()

In [21]:
metric = keras.metrics.MeanSquaredLogarithmicError()

In [22]:
optim = keras.optimizers.Adagrad()

In [23]:
def create_model(layers,activation):
  model = Sequential()
  for i,nodes in enumerate(layers):
    if i==0:
      model.add(Dense(nodes,input_dim = x_train.shape[1]))
      model.add(Activation(activation))
      model.add(Dropout(0.3))
    else:
      model.add(Dense(nodes))
      model.add(Activation(activation))
      model.add(Dropout(0.3))
  
  model.add(Dense(1,activation = "linear"))
  model.compile(optimizer = optim,loss = mle,metrics = [metric])
  return model

In [24]:
from keras.wrappers.scikit_learn import KerasRegressor

In [25]:
from sklearn.model_selection import GridSearchCV

In [26]:
model = KerasRegressor(create_model,verbose = 0)

In [27]:
layers = [[20],[64,32,16],[128,64,32,16,4]]
activations = ["relu","selu","softplus"]
param_grid = dict(layers = layers,activation = activations,batch_size = [64,128,256],epochs = [10,50,100])
grid = GridSearchCV(model,param_grid=param_grid,cv = 5)

In [28]:
grid_history = grid.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [29]:
grid_history.best_score_

-0.022188999322282167

In [30]:
grid_history.best_params_

{'activation': 'relu', 'batch_size': 64, 'epochs': 10, 'layers': [64, 32, 16]}